# Nested Sampling Algorithm

- Jeffrey's prior for $D_0$
- Uniform Priors for $\alpha$, $f$ and $\sigma^2_{mn}$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import scipy.stats as stats
import warnings

In [2]:
def jeff(x,magmin, magmax):
    return magmin*exp(u*log(magmax/magmin))
def uni(x, magmin, magmax):
    return 1/(magmax, magmin)


## likelihood calc function

In [3]:
@jit
def noisylikelihoodcalc(posn, DT, D, ALPHA, F =0 , varmn = 0):
    # time series
    n = len(posn)

    #init value
    logl = 0
    k_BT =1
    

    # Intial values of the variance mean both noisy and clean
    mob = D * (np.abs(posn[0]) ** ALPHA)
    std_dev_noise = 2 * mob * DT + 2 * varmn
    noisy_mean = posn[0] + (mob * F + k_BT * (ALPHA * D * ((np.abs(posn[0]) ** (ALPHA - 1)) * (np.sign(posn[0]))))) * DT
    
    # log likelihood at 1st step
    logl = logl - ((posn[1] - noisy_mean) ** 2) / (2 * std_dev_noise) - np.log(np.sqrt(2 * np.pi * std_dev_noise))
    
    for i in range(2,n):
        # Mobility at the step
        mob = D * np.abs(posn[i - 1]) ** ALPHA

        # clean variance at the jth step
        std_dev = 2 * mob * DT

        # clean mean position at the jth time step
        mean_dist = posn[i - 1] + (mob * F + ALPHA * D * (np.abs(posn[i - 1]) ** (ALPHA - 1)) * (np.sign(posn[i - 1]))) * DT

        # noisy mean position at the jth step
        noisy_mean = mean_dist - varmn/(std_dev_noise) * (posn[i-1] - noisy_mean)

        # noisy variance at the jth posn
        std_dev_noise = std_dev + varmn * (2 - varmn / std_dev_noise)

        # calculation of log likelihood
        logl = logl - ((posn[i] - noisy_mean) ** 2) / (2 * std_dev_noise) - 0.5 * np.log(2 * np.pi * std_dev_noise)
    
    return logl

## Model for calculating Evidence and with consider Model parameters

Instead of defining different functions for different models we could use the theta_max and theta_min vectors to define the parameters properly.

In [4]:
@jit
def Evidence(data, theta, npoints, tol):
    logZ = 0
    walker = walkergen(npoints, theta_max, theta_min)
    for i in range(0,npoints):
        ll[i] = noisylikelihoodcalc(p_data, 1, walker[i][0], walker[i][1], walker[i][2], walker[i][3])
    
    

In [5]:
@jit
def walkergen(n, tmax, tmin):
    x = np.zeros((n,4))
    for i in range(0,n):
        for j in range(0,4):
            x[i][j] = np.random.uniform(tmin[j],tmax[j])
    return x

In [6]:
# @jit 
def logsum(a, b):
    return max(a,b) + np.log(1+ np.exp(-np.abs(a-b)))

## $\theta_{max}$ and $\theta_{min}$ vectors

we use this to define the priors

In [7]:
D_min = 10**(-4)
D_max = 10**2
# D_min = 0.
# D_max = 1.


f_min = -1.
f_max = 1.

alpha_min = -2.
alpha_max = 2.

varmn_min = 1.
varmn_max = 100.

theta_max_pc = np.array([D_max, alpha_max, f_max, 0])
theta_min_pc = np.array([D_min, alpha_min, f_min, 0])

theta_max_fc = np.array([D_max, alpha_max, 0, 0])
theta_min_fc = np.array([D_min, alpha_min, 0, 0])

theta_max_pmn = np.array([D_max, alpha_max, f_max, varmn_max])
theta_min_pmn = np.array([D_min, alpha_min, f_min, varmn_min])

theta_max_fmn = np.array([D_max, alpha_max, 0, varmn_max])
theta_min_fmn = np.array([D_min, alpha_min, 0, varmn_min])


In [41]:
# p_data = np.array([50,46.743,46.751,40.915,45.066,50.56,44.448,40.518,40.145,39.551,42.887,40.218,44.399,38.416,36.626,35.867,30.252,28.843,26.412,27.815,28.166,32.899,34.738,33.237,24.448,22.771,19.285,24.966,23.912,23.839,23.83,22.253,15.661,20.156,20.166,16.9,15.495,18.734,18.234,17.823,16.052,11.424,9.4223,7.3746,7.2549,7.2847,6.7366,7.7324,6.8572,7.034,5.3564,3.7969,3.4073,4.3653,4.56,4.8884,3.8021,2.9259,3.5701,4.0879,2.3471,0.95298,1.7231,1.7968,1.753,2.1651,2.1354,2.404,2.9188,1.1974,1.3358,1.428,1.8476,2.1991,1.1013,1.3227,0.71634,0.68376,0.89795,1.6167,2.0934,2.4119,1.0693,1.2777,2.2396,0.30283,0.64551,-0.44319,-0.79814,-0.82102,-2.0478,-1.6834,-1.8176,-1.3092,-1.2355,-1.8231,-1.8543,-2.1829,-4.2785,-4.2926])


# my data
# p_data = np.array([50.   , 43.235, 36.248, 30.705, 31.312, 28.791, 26.838, 26.137,       24.245, 23.097, 21.414, 19.042, 20.38 , 20.536, 18.37 , 19.826,       22.181, 24.422, 23.84 , 20.125, 18.406, 21.642, 19.997, 17.496,       17.334, 19.815, 15.026, 15.195, 12.337, 15.354, 19.697, 20.12 ,       16.174, 14.4  , 13.479, 17.593, 15.508, 14.726, 11.388,  9.644,        9.469,  9.747, 11.076,  8.236,  8.134,  8.891,  7.284,  7.751,        6.388,  5.29 ,  4.28 ,  3.221,  2.371,  2.023,  1.591,  2.872,        4.703,  5.233,  4.382,  4.458,  3.87 ,  5.508,  4.44 ,  5.594,        6.053,  7.122,  8.703,  6.282,  6.946,  3.005,  1.706,  2.829,        2.391,  1.722,  0.27 ,  0.477,  0.391,  0.22 ,  0.469,  1.013,        1.807,  2.978,  3.748,  2.164,  3.568,  4.455,  3.843,  4.115,        4.108,  1.415,  2.297,  1.828,  0.968,  0.686,  0.648,  1.19 ,        1.096,  0.899,  1.504,  2.276])

p_data = np.array([50.   , 51.367, 48.699, 44.642, 43.553, 49.184, 49.222, 50.955,       38.148, 46.938, 34.223, 38.415, 36.054, 35.257, 35.473, 29.603,       34.158, 28.346, 33.686, 31.855, 32.454, 36.521, 31.482, 35.63 ,       39.712, 38.944, 37.343, 39.255, 35.665, 31.73 , 28.254, 30.081,       35.46 , 34.247, 32.114, 30.253, 27.704, 27.316, 32.169, 32.617,       27.722, 32.482, 34.431, 28.408, 27.382, 28.881, 29.671, 29.079,       31.088, 33.544, 34.063, 39.901, 38.426, 49.951, 50.612, 54.932,       54.549, 60.98 , 60.88 , 59.873, 64.145, 65.389, 57.596, 49.069,       50.313, 49.782, 44.892, 56.176, 53.266, 52.5  , 43.84 , 38.461,       38.153, 38.319, 34.644, 32.851, 34.693, 31.565, 29.909, 31.77 ,       27.628, 24.255, 20.444, 24.825, 28.336, 30.011, 28.37 , 21.392,       17.319, 18.046, 16.723, 16.915, 19.138, 18.129, 17.593, 14.156,       16.958, 13.758, 17.603, 17.622])

# MCMC walker

Tried my hand at coding a MCMC walker using the Metropolis Hastings algo. The proposal function is a standard gaussian. To make it compatible for different models use can use different step sizes and then set the step sizes for the non fixed parameters to be the same and step size for the fixed parameter to be 0. So the walker is also constrained to walk over a surface that has fixed parameter of the model to be fixed. Setting the proposal function like this means we can define an easier code snippet without having to define multiple methods for different models and then it would be easier to define.

We could also go an easier way and then generate the walkers only in the constrained surface

Have to try both, but have a feeling that the secind method will be more efficient.

In [9]:
# @jit
def MCMCwalker(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in range(0,4):
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [10]:
# @jit
def MCMCwalkerfmn(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in [0,1,3]:
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [48]:
# @jit
def MCMCwalkerpc(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in range(0,3):
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar and prop<theta_max_pc and prop >theta_min_pc:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [12]:
# @jit
def MCMCwalkerfc(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in [0,1]:
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

# Nested Sampling Code starts here

In [42]:
# steps for the random walkers in the MCMC algo
Nsteps = 18000

# no of walkers considered
npoints = 100

# init likelihood of pull, clean and free, clean models
llpc = np.zeros(npoints)
llfc = np.zeros(npoints)
llpmn = np.zeros(npoints)
llfmn = np.zeros(npoints)

# walkers pull clean and walker free clean
walkers_pc = np.array(walkergen(npoints, theta_max_pc,theta_min_pc))
walkers_fc = np.array(walkergen(npoints, theta_max_fc,theta_min_fc))
walkers_pmn = np.array(walkergen(npoints, theta_max_pmn,theta_min_pmn))
walkers_fmn = np.array(walkergen(npoints, theta_max_fmn,theta_min_fmn))

# likelihood  of Mpc and Mfc
for i in range(0,npoints):
    llpc[i] = noisylikelihoodcalc(p_data, 1, *walkers_pc[i])
    llfc[i] = noisylikelihoodcalc(p_data, 1, *walkers_fc[i])
    llpmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_pmn[i])
    llfmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_fmn[i])


# wt = 1/(npoints + 1)
logwt = -np.log(npoints + 1)

#Zpc is Z pull clean
#Zfc is Z free clean

# these are the remainders in the jth iteration
logZpcrem = np.log(0)
logZfcrem = np.log(0)
logZpmnrem = np.log(0)
logZfmnrem = np.log(0)

# these are the Log Evidences for the respective models
logZpc = np.log(0)
logZfc = np.log(0)
logZpmn = np.log(0)
logZfmn = np.log(0)


logp_pc = min(llpc) + logwt
logp_fc = min(llfc) + logwt
logp_pmn = min(llpmn) + logwt
logp_fmn = min(llfmn) + logwt


logZpmn = logsum(logZpmn,logp_pmn)
logZfmn = logsum(logZfmn,logp_fmn)
logZpc = logsum(logZpc,logp_pc)
logZfc = logsum(logZfc,logp_fc)


for i in range(0, npoints):
    logZfcrem = logsum(llfc[i], logZfcrem)
    logZpcrem = logsum(llpc[i], logZpcrem)
    logZfmnrem = logsum(llfmn[i], logZfmnrem)
    logZpmnrem = logsum(llpmn[i], logZpmnrem)

logZfcrem = logZfcrem + logwt
logZpcrem = logZpcrem + logwt
logZfmnrem = logZfmnrem + logwt
logZpmnrem = logZpmnrem + logwt


logZpc,min(llpc),logZfc,min(llfc),logZpmn,min(llpmn),logZfmn,min(llfmn)

/tmp/ipykernel_109434/3447937050.py:34: RuntimeWarning: divide by zero encountered in log
  logZpcrem = np.log(0)
/tmp/ipykernel_109434/3447937050.py:35: RuntimeWarning: divide by zero encountered in log
  logZfcrem = np.log(0)
/tmp/ipykernel_109434/3447937050.py:36: RuntimeWarning: divide by zero encountered in log
  logZpmnrem = np.log(0)
/tmp/ipykernel_109434/3447937050.py:37: RuntimeWarning: divide by zero encountered in log
  logZfmnrem = np.log(0)
/tmp/ipykernel_109434/3447937050.py:40: RuntimeWarning: divide by zero encountered in log
  logZpc = np.log(0)
/tmp/ipykernel_109434/3447937050.py:41: RuntimeWarning: divide by zero encountered in log
  logZfc = np.log(0)
/tmp/ipykernel_109434/3447937050.py:42: RuntimeWarning: divide by zero encountered in log
  logZpmn = np.log(0)
/tmp/ipykernel_109434/3447937050.py:43: RuntimeWarning: divide by zero encountered in log
  logZfmn = np.log(0)


(-1711742.1780461695,
 -1711737.5629256526,
 -16487.984086600423,
 -16483.36896608358,
 -911534.1150897283,
 -911529.4999692114,
 -7532.710046003649,
 -7528.094925486807)

In [43]:
# initial stepsize
step = 0

# int index of the loop
j=1

# tolerance value for stopping condition
tol = np.log(10**(-4))

# lowest likelihood in each iteration
lstarpc = []

# log weight
logwt = -np.log(npoints + 1)

# posterior avergae and posterior samples
lpostavgpc = np.zeros(4)
lpostpc = np.zeros(4)

# posterior 2nd moment and posterior 2nd moment samples
post2mavgpc = np.zeros(4)
post2mpc = np.zeros(4)

#information to calculate error
Hpc = 0

#stopping ratio calculator
Rpc = logZpcrem - logZpc

if Rpc < tol:
    print("term1 stop")
    print(logZpc)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglpc = [noisylikelihoodcalc(p_data, 1, *walkers_pc[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarpc.append(min(loglpc))

    # walker index with minimum loglikelihood
    ppc = np.argmin(loglpc)

    # previous iterations log Evidence
    prevlogZpc = logZpc

    # previous iteration's H(information)
    prevHpc = Hpc
    
    # log Z finding
    logZpc = logsum(logZpc, (lstarpc[-1] + logwt))

    # posterior sample
    lpostpc = (walkers_pc[ppc])*np.exp(lstarpc[-1])*np.exp(logwt)

    #information calculation
    Hpc += np.exp(logwt)*np.exp(lstarpc[-1])*lstarpc[-1]

    print("Information")
    print(Hpc)

    # posterior 2nd moment sample
    post2mpc = ((walkers_pc[ppc]) ** 2) * np.exp(lstarpc[-1]) * np.exp(logwt)

    # posterior summation after each iteration
    lpostavgpc = lpostavgpc + lpostpc

    # posterior 2nd moment average
    post2mavgpc = post2mavgpc + post2mpc
    
    print("posterior");
    print(lpostavgpc);
    
    print("Evidence");
    print(logZpc);

    # init Zremaining for the jth iteration
    logZpcrem = np.log(0)

    # calculate Zremaining for the jth iteration
    for i in range(0, npoints):
        logZpcrem = logsum(loglpc[i], logZpcrem)
        
    logZpcrem = logZpcrem + logwt

    print( "Zrem =  "+str(logZpcrem));
    print("lowest likelihood = "+str(min(loglpc)))

    # ratio that determines the stopping ratio
    Rpc = logZpcrem - logZpc
    
    print("ratio")
    print(Rpc)

    #break if stopping ratio is less than tolerance
    if Rpc < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_pc[newwalkernumber]

    # walker replacement  
    trace = MCMCwalkerpc(p_data, 1, thetawalker, lstarpc[-1] , Nsteps, step)

    # step modulation in MCMC walker
    # step = trace[0]
    step = 1

    # replacing the worst walker with the copied walker that undergoes a random walk
    walkers_pc[ppc] = trace[1]

# final evidence calulcation
LogfinalZpc = logsum(prevlogZpc, logZpcrem)

LogfinalZpc

Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-1711741.4898617782
Zrem =  -296.3814685948132
lowest likelihood = -1711737.5629256526
ratio
1711445.1083931834
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-443075.54854203
Zrem =  -293.92149817872945
lowest likelihood = -443070.9135208514
ratio
442781.62704385124
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-330950.04444433324
Zrem =  -291.69146415094144
lowest likelihood = -330945.39947282383
ratio
330658.3529801823


/tmp/ipykernel_109434/3929159730.py:81: RuntimeWarning: divide by zero encountered in log
  logZpcrem = np.log(0)


Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-296913.8526573238
Zrem =  -282.9127359186056
lowest likelihood = -296909.1977354836
ratio
296630.9399214052
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-261095.828227599
Zrem =  -282.92267054082316
lowest likelihood = -261091.1633554279
ratio
260812.90555705817
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-209557.7460251809
Zrem =  -282.9325362353624
lowest likelihood = -209553.07120267896
ratio
209274.81348894554
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-197871.43851340422
Zrem =  -282.9424742139459
lowest likelihood = -197866.7537405714
ratio
197588.49603919027
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-178869.3732410803
Zrem =  -282.9523976723304
lowest likelihood = -178864.67851791662
ratio
178586.42084340795
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-166899.4199285092
Zrem =  -282.9623097253814
lowest likelihood = -166894.71525501466
ratio
166616.4576187838
Information
0.0
posterior
[0. 0. 0. 0.]
Evid

KeyboardInterrupt: 

In [33]:
# init posterior avg at the jmax iteration
lapc = np.zeros(4)
l2pc = np.zeros(4)
lh = 0
Hpcfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lapc += walkers_pc[i] * np.exp(llpc[i])
    l2pc += (walkers_pc[i]**2) * np.exp(llpc[i])
    lh += llpc[i]*np.exp(llpc[i])
    
lapc *= np.exp(logwt)
l2pc *= np.exp(logwt)
lh *= np.exp(logwt)

# final posterior average
postfinpc = lpostavgpc+lapc

# final posterior 2 moment
post2mfinpc = post2mavgpc+l2pc

# dividing by the evidence of the model to find the posterior
normpostfinpc = postfinpc/np.exp(LogfinalZpc)

# normalizing posterior 2nd moment
normpost2mfinpc = post2mfinpc/np.exp(LogfinalZpc)

# calculating final information
Hpcfin = prevHpc + lh
Hpcfin = Hpcfin/np.exp(LogfinalZpc) - LogfinalZpc
logZerrpc = np.sqrt(Hpcfin/npoints)

# variance and error in the parameters
varpc = normpost2mfinpc - normpostfinpc**2
stddevpc = np.sqrt(varpc)

# printing information
print("Posterior theta = "+ str(normpostfinpc)+" Error = "+str(stddevpc))
print("ln(Evidence) = "+str(LogfinalZpc)+" +/- "+str(logZerrpc))

Posterior theta = [ 0.17623712  0.97882937 -0.35999985  0.        ] Error = [0.02173898 0.05708007 0.05926929 0.        ]
ln(Evidence) = -176.41925133027257 +/- 0.1285233720528637


## Posterior Value using $Model_{pull, clean}$

The Log Evidence is given as:

$$\ln(Z) = 183.9 \pm 0.21$$

The Posterior averages are given as:
- $D_0 = 0.169 \pm 0.004$
- $\alpha = 0.99 \pm 0.024 $
- $f = -0.29 \pm 0.03$

In [34]:
#initial step size
step = 1

# init index
j=1

#tolerance for stopping condition
tol = np.log(10**(-4))

# lowest likelihood for the jth iteration
lstarfc = []

# log wth value for prior mass
logwt = -np.log(npoints + 1)

# posterior average and posterior samples
lpostavgfc = np.zeros(4)
lpostfc = np.zeros(4)

#information
Hfc = 0

# posterior 2nd moment and posterior 2nd moment samples
post2mavgfc = np.zeros(4)
post2mfc = np.zeros(4)

# stopping ratio
Rfc = logZfcrem - logZfc

# stopping condition
if Rfc< tol:
    print(logZfc)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglfc = [noisylikelihoodcalc(p_data, 1, *walkers_fc[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarfc.append(min(loglfc))

    pfc = np.argmin(loglfc)

    # storing the evidence of the previous iteration
    prevlogZfc = logZfc

    #storing the information of the previous iteration
    prevHfc = Hfc
    
    # log Z finding
    logZfc = logsum(logZfc, (lstarfc[-1] + logwt))

    # posterior samples
    lpostfc = (walkers_fc[pfc])*np.exp(lstarfc[-1])*np.exp(logwt)

    #information
    Hfc += np.exp(logwt)*np.exp(lstarfc[-1])*lstarfc[-1]

    #posterior 2m samples
    post2mfc = ((walkers_fc[pfc])**2)*np.exp(lstarfc[-1])*np.exp(logwt)
    
    # posterior average
    lpostavgfc = lpostavgfc + lpostfc

    #posterior 2m average
    post2mavgfc = post2mavgfc + post2mfc

    print("posterior");
    print(lpostavgfc);
    
    print("Evidence");
    print(logZfc);

    # init Z remaining of the jth iteration
    logZfcrem = np.log(0)

    # calculating Z remaning of the jth iteration
    for i in range(0, npoints):
        logZfcrem = logsum(loglfc[i], logZfcrem)
        
    logZfcrem = logZfcrem + logwt

    print( "Zrem =  "+str(logZfcrem));
    print("lowest likelihood = "+str(min(loglfc)))

    # ratio that determines the stopping ratio
    Rfc = logZfcrem - logZfc
    
    print("ratio")
    print(Rfc)

    #stopping condition and breaks loop
    if Rfc < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    # thetawalker = np.array(walkers_fc[newwalkernumber])

    # walker replacement  
    trace = MCMCwalkerfc(p_data, 1, thetawalker, lstarfc[-1] , Nsteps, 1)

    # step modulation for the random walk
    step = 1

    # replacement of the worst walker with walker performing random walk
    walkers_fc[pfc] = trace[1]
    walkers_fc[pfc][2] = 0
    walkers_fc[pfc][3] = 0

#final Z calculation
LogfinalZfc = logsum(prevlogZfc, logZfcrem)

LogfinalZfc

posterior
[0. 0. 0. 0.]
Evidence
-51675.00243445784
Zrem =  -200.52440943661082
lowest likelihood = -51671.07549833222
ratio
51474.47802502123
posterior
[0. 0. 0. 0.]
Evidence
-32460.297671274362
Zrem =  -184.71553223310386
lowest likelihood = -32455.662650095815
ratio
32275.582139041257
posterior
[0. 0. 0. 0.]
Evidence
-29783.023670876017
Zrem =  -184.02250288611307
lowest likelihood = -29778.378699366614
ratio
29599.001167989903
posterior
[0. 0. 0. 0.]
Evidence
-23645.981790949656
Zrem =  -183.62192206549847
lowest likelihood = -23641.3268691094
ratio
23462.359868884156


/tmp/ipykernel_89947/1927483834.py:77: RuntimeWarning: divide by zero encountered in log
  logZfcrem = np.log(0)


posterior
[0. 0. 0. 0.]
Evidence
-22257.740596864252
Zrem =  -183.3430885884489
lowest likelihood = -22253.075724693146
ratio
22074.397508275804
posterior
[0. 0. 0. 0.]
Evidence
-18106.31640647648
Zrem =  -183.1367369403404
lowest likelihood = -18101.641583974517
ratio
17923.179669536137
posterior
[0. 0. 0. 0.]
Evidence
-15486.836954209924
Zrem =  -182.96451384095874
lowest likelihood = -15482.15218137711
ratio
15303.872440368965
posterior
[0. 0. 0. 0.]
Evidence
-13259.09146120147
Zrem =  -182.8164411771825
lowest likelihood = -13254.396738037804
ratio
13076.275020024286
posterior
[0. 0. 0. 0.]
Evidence
-11389.973156059854
Zrem =  -182.6906160255866
lowest likelihood = -11385.268482565334
ratio
11207.282540034268
posterior
[0. 0. 0. 0.]
Evidence
-9551.30810259473
Zrem =  -182.5822001925771
lowest likelihood = -9546.593478769357
ratio
9368.725902402153
posterior
[0. 0. 0. 0.]
Evidence
-9328.5120329517
Zrem =  -182.48532374913418
lowest likelihood = -9323.787458795474
ratio
9146.02670920

-181.09568608993715

In [35]:
# init posterior avg at the jmax iteration
lafc = np.zeros(4)
l2fc = np.zeros(4)
lahfc = 0

Hfcfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lafc += walkers_fc[i] * np.exp(llfc[i])
    l2fc += (walkers_fc[i]**2) * np.exp(llfc[i])
    lahfc += llfc[i]*np.exp(llfc[i])
    
lafc *= np.exp(logwt)
l2fc *= np.exp(logwt)
lahfc *= np.exp(logwt)

#final posterior calculation
postfinfc = lpostavgfc+lafc
post2mfinfc = post2mavgfc+l2fc

# dividing the posterior by the evidence
normpostfinfc = postfinfc/np.exp(LogfinalZfc)
normpost2mfinfc = post2mfinfc/np.exp(LogfinalZfc)

# information final calculation
Hfcfin = prevHfc + lahfc
Hfcfin = Hfcfin/np.exp(LogfinalZfc) - LogfinalZfc
logZerrfc = np.sqrt(Hfcfin/npoints)

# variance and eror of posterior parameters
varfc = normpost2mfinfc - normpostfinfc**2
stddevfc = np.sqrt(varfc)

# printing information
print("Posterior theta = "+ str(normpostfinfc)+" Error = "+str(stddevfc))
print("ln(Evidence) = "+str(LogfinalZfc)+" +/- "+str(logZerrfc))

Posterior theta = [0.16671055 1.01927168 0.         0.        ] Error = [0.00752344 0.02378664 0.         0.        ]
ln(Evidence) = -181.09568608993715 +/- 0.10115919393938466


# Nested sampling code for noisy models

## commented code

In [111]:
# # steps for the random walkers in the MCMC algo
# Nsteps = 10000

# # no of walkers considered
# npoints = 100

# # init likelihood of pull, clean and free, clean models
# llpmn = np.zeros(npoints)
# llfmn = np.zeros(npoints)

# # walkers pull clean and walker free clean
# walkers_pmn = np.array(walkergen(npoints, theta_max_pmn,theta_min_pmn))
# walkers_fmn = np.array(walkergen(npoints, theta_max_fmn,theta_min_fmn))

# # print(walkers_fmn)
# # print(walkers_pmn)

# # likelihood  of Mpmn and Mfmn
# for i in range(0,npoints):
#     llpmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_pmn[i])
#     llfmn[i] = noisylikelihoodcalc(p_data, 1, *walkers_fmn[i])


# # wt = 1/(npoints + 1)
# logwt = -np.log(npoints + 1)

# #Zpmn is Z pull clean
# #Zfmn is Z free clean

# # these are the remainders in the jth iteration
# logZpmnrem = np.log(0)
# logZfmnrem = np.log(0)

# # these are the Log Evidences for the respective models
# logZpmn = np.log(0)
# logZfmn = np.log(0)


# logp_pmn = min(llpmn) + logwt
# logp_fmn = min(llfmn) + logwt

# logZpmn = logsum(logZpmn,logp_pmn)
# logZfmn = logsum(logZfmn,logp_fmn)


# for i in range(0, npoints):
#     logZfmnrem = logsum(llfmn[i], logZfmnrem)
#     logZpmnrem = logsum(llpmn[i], logZpmnrem)

# logZfmnrem = logZfmnrem + logwt
# logZpmnrem = logZpmnrem + logwt

# logZpmn,min(llpmn),logZfmn,min(llfmn)

/tmp/ipykernel_63131/1179302201.py:31: RuntimeWarning: divide by zero encountered in log
  logZpmnrem = np.log(0)
/tmp/ipykernel_63131/1179302201.py:32: RuntimeWarning: divide by zero encountered in log
  logZfmnrem = np.log(0)
/tmp/ipykernel_63131/1179302201.py:35: RuntimeWarning: divide by zero encountered in log
  logZpmn = np.log(0)
/tmp/ipykernel_63131/1179302201.py:36: RuntimeWarning: divide by zero encountered in log
  logZfmn = np.log(0)


(-316139.2501793184,
 -316134.63505880156,
 -12067.878821259246,
 -12063.263700742405)

## commented code end

In [36]:
# initial stepsize
step = 1

# int index of the loop
j=1

# tolerance value for stopping condition
tol = np.log(10**(-4))

# lowest likelihood in each iteration
lstarpmn = []

# log weight
logwt = -np.log(npoints + 1)

# posterior avergae and posterior samples
lpostavgpmn = np.zeros(4)
lpostpmn = np.zeros(4)

# posterior 2nd moment and posterior 2nd moment samples
post2mavgpmn = np.zeros(4)
post2mpmn = np.zeros(4)

#information to calculate error
Hpmn = 0

#stopping ratio calculator
Rpmn = logZpmnrem - logZpmn

if Rpmn < tol:
    print("term1 stop")
    print(logZpmn)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglpmn = [noisylikelihoodcalc(p_data, 1, *walkers_pmn[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarpmn.append(min(loglpmn))

    # walker index with minimum loglikelihood
    ppmn = np.argmin(loglpmn)

    # previous iterations log Evidence
    prevlogZpmn = logZpmn

    # previous iteration's H(information)
    prevHpmn = Hpmn
    
    # log Z finding
    logZpmn = logsum(logZpmn, (lstarpmn[-1] + logwt))

    # posterior sample
    lpostpmn = (walkers_pmn[ppmn])*np.exp(lstarpmn[-1])*np.exp(logwt)

    #information calculation
    Hpmn += np.exp(logwt)*np.exp(lstarpmn[-1])*lstarpmn[-1]

    print("Information")
    print(Hpmn)

    # posterior 2nd moment sample
    post2mpmn = ((walkers_pmn[ppmn]) ** 2) * np.exp(lstarpmn[-1]) * np.exp(logwt)

    # posterior summation after each iteration
    lpostavgpmn = lpostavgpmn + lpostpmn

    # posterior 2nd moment average
    post2mavgpmn = post2mavgpmn + post2mpmn
    
    print("posterior");
    print(lpostavgpmn);
    
    print("Evidence");
    print(logZpmn);

    # init Zremaining for the jth iteration
    logZpmnrem = np.log(0)

    # calculate Zremaining for the jth iteration
    for i in range(0, npoints):
        logZpmnrem = logsum(loglpmn[i], logZpmnrem)
        
    logZpmnrem = logZpmnrem + logwt

    print( "Zrem =  "+str(logZpmnrem));
    print("lowest likelihood = "+str(min(loglpmn)))

    # ratio that determines the stopping ratio
    Rpmn = logZpmnrem - logZpmn
    
    print("ratio")
    print(Rpmn)

    #break if stopping ratio is less than tolerance
    if Rpmn < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_pmn[newwalkernumber]

    # walker replacement  
    trace = MCMCwalker(p_data, 1, thetawalker, lstarpmn[-1] , Nsteps, step)

    # step modulation in MCMC walker
    step = 1

    # replacing the worst walker with the copied walker that undergoes a random walk
    walkers_pmn[ppmn] = trace[1]

# final evidence calulcation
LogfinalZpmn = logsum(prevlogZpmn, logZpmnrem)

LogfinalZpmn

Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-462471.41658986354
Zrem =  -336.6341768427552
lowest likelihood = -462467.4896537379
ratio
462134.7824130208
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-269699.1427193783
Zrem =  -336.6441271736084
lowest likelihood = -269694.5076981997
ratio
269362.4985922047


/tmp/ipykernel_89947/178806207.py:81: RuntimeWarning: divide by zero encountered in log
  logZpmnrem = np.log(0)


Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-70917.98841861013
Zrem =  -336.65407750446155
lowest likelihood = -70913.34344710072
ratio
70581.33434110567
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-55008.99826034883
Zrem =  -311.7985709966765
lowest likelihood = -55004.34333850858
ratio
54697.19968935216
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-50969.08909823176
Zrem =  -311.8085213275297
lowest likelihood = -50964.42422606065
ratio
50657.28057690423
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-45663.27205589008
Zrem =  -311.81847165838286
lowest likelihood = -45658.597233388115
ratio
45351.45358423169
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-41493.43851563454
Zrem =  -293.92355652662343
lowest likelihood = -41488.753742801724
ratio
41199.51495910792
Information
0.0
posterior
[0. 0. 0. 0.]
Evidence
-35070.8900713377
Zrem =  -293.9335068574766
lowest likelihood = -35066.195348174035
ratio
34776.95656448022
Information
0.0
posterior
[0. 0. 0. 0.]
Evidenc

-178.82108277497056

In [37]:
# init posterior avg at the jmax iteration
lapmn = np.zeros(4)
l2pmn = np.zeros(4)
lhmn = 0
Hpmnfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lapmn += walkers_pmn[i] * np.exp(llpmn[i])
    l2pmn += (walkers_pmn[i]**2) * np.exp(llpmn[i])
    lh += llpmn[i]*np.exp(llpmn[i])
    
lapmn *= np.exp(logwt)
l2pmn *= np.exp(logwt)
lhmn *= np.exp(logwt)

# final posterior average
postfinpmn = lpostavgpmn+lapmn

# final posterior 2 moment
post2mfinpmn = post2mavgpmn+l2pmn

# dividing by the evidence of the model to find the posterior
normpostfinpmn = postfinpmn/np.exp(LogfinalZpmn)

# normalizing posterior 2nd moment
normpost2mfinpmn = post2mfinpmn/np.exp(LogfinalZpmn)

# calculating final information
Hpmnfin = prevHpmn + lhmn
Hpmnfin = Hpmnfin/np.exp(LogfinalZpmn) - LogfinalZpmn
logZerrpmn = np.sqrt(Hpmnfin/npoints)

# variance and error in the parameters
varpmn = normpost2mfinpmn - normpostfinpmn**2
stddevpmn = np.sqrt(varpmn)

# printing information
print("Posterior theta = "+ str(normpostfinpmn)+" Error = "+str(stddevpmn))
print("ln(Evidence) = "+str(LogfinalZpmn)+" +/- "+str(logZerrpmn))

Posterior theta = [ 0.23175368  0.88687374 -0.34605166 -0.02524971] Error = [0.06503723 0.10861215 0.10439731 0.03442744]
ln(Evidence) = -178.82108277497056 +/- 0.19897079472243728


## Posterior Value using $Model_{pull, clean}$

The Log Evidence is given as:

$$\ln(Z) = 183.9 \pm 0.21$$

The Posterior averages are given as:
- $D_0 = 0.169 \pm 0.004$
- $\alpha = 0.99 \pm 0.024 $
- $f = -0.29 \pm 0.03$

In [38]:
#initial step size
step = 1

# init index
j=1

#tolerance for stopping condition
tol = np.log(10**(-4))

# lowest likelihood for the jth iteration
lstarfmn = []

# log wth value for prior mass
logwt = -np.log(npoints + 1)

# posterior average and posterior samples
lpostavgfmn = np.zeros(4)
lpostfmn = np.zeros(4)

#information
Hfmn = 0

# posterior 2nd moment and posterior 2nd moment samples
post2mavgfmn = np.zeros(4)
post2mfmn = np.zeros(4)

# stopping ratio
Rfmn = logZfmnrem - logZfmn

# stopping condition
if Rfmn< tol:
    print(logZfmn)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglfmn = [noisylikelihoodcalc(p_data, 1, *walkers_fmn[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarfmn.append(min(loglfmn))

    pfmn = np.argmin(loglfmn)

    # storing the evidence of the previous iteration
    prevlogZfmn = logZfmn

    #storing the information of the previous iteration
    prevHfmn = Hfmn
    
    # log Z finding
    logZfmn = logsum(logZfmn, (lstarfmn[-1] + logwt))

    # posterior samples
    lpostfmn = (walkers_fmn[pfmn])*np.exp(lstarfmn[-1])*np.exp(logwt)

    #information
    Hfmn += np.exp(logwt)*np.exp(lstarfmn[-1])*lstarfmn[-1]

    #posterior 2m samples
    post2mfmn = ((walkers_fmn[pfmn])**2)*np.exp(lstarfmn[-1])*np.exp(logwt)
    
    # posterior average
    lpostavgfmn = lpostavgfmn + lpostfmn

    #posterior 2m average
    post2mavgfmn = post2mavgfmn + post2mfmn

    print("posterior");
    print(lpostavgfmn);
    
    print("Evidence");
    print(logZfmn);

    # init Z remaining of the jth iteration
    logZfmnrem = np.log(0)

    # calculating Z remaning of the jth iteration
    for i in range(0, npoints):
        logZfmnrem = logsum(loglfmn[i], logZfmnrem)
        
    logZfmnrem = logZfmnrem + logwt

    print( "Zrem =  "+str(logZfmnrem));
    print("lowest likelihood = "+str(min(loglfmn)))

    # ratio that determines the stopping ratio
    Rfmn = logZfmnrem - logZfmn
    
    print("ratio")
    print(Rfmn)

    #stopping condition and breaks loop
    if Rfmn < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_fmn[newwalkernumber]

    # walker replacement  
    trace = MCMCwalkerfmn(p_data, 1, thetawalker, lstarfmn[-1] , Nsteps, step)

    # step modulation for the random walk
    step = 1

    # replacement of the worst walker with walker performing random walk
    walkers_fmn[pfmn] = trace[1]

#final Z calculation
LogfinalZfmn = logsum(prevlogZfmn, logZfmnrem)

LogfinalZfmn

posterior
[0. 0. 0. 0.]
Evidence
-41262.79775146131
Zrem =  -280.5232016385981
lowest likelihood = -41258.870815335686
ratio
40982.27454982271
posterior
[0. 0. 0. 0.]
Evidence
-25868.05895286985
Zrem =  -185.28675993637412
lowest likelihood = -25863.423931691304
ratio
25682.772192933477


/tmp/ipykernel_89947/970416612.py:77: RuntimeWarning: divide by zero encountered in log
  logZfmnrem = np.log(0)


posterior
[0. 0. 0. 0.]
Evidence
-25010.79166659128
Zrem =  -184.6718697073681
lowest likelihood = -25006.14669508188
ratio
24826.119796883915
posterior
[0. 0. 0. 0.]
Evidence
-24158.208623042447
Zrem =  -184.68182003822128
lowest likelihood = -24153.553701202192
ratio
23973.526803004224
posterior
[0. 0. 0. 0.]
Evidence
-18472.266579283558
Zrem =  -184.69177036907445
lowest likelihood = -18467.60170711245
ratio
18287.574808914484
posterior
[0. 0. 0. 0.]
Evidence
-14126.006968111442
Zrem =  -184.70172069992762
lowest likelihood = -14121.332145609482
ratio
13941.305247411514
posterior
[0. 0. 0. 0.]
Evidence
-13871.980262953724
Zrem =  -182.98222810802983
lowest likelihood = -13867.29549012091
ratio
13688.998034845694
posterior
[0. 0. 0. 0.]
Evidence
-13490.874126413231
Zrem =  -182.992178438883
lowest likelihood = -13486.179403249565
ratio
13307.881947974349
posterior
[0. 0. 0. 0.]
Evidence
-13110.76127637841
Zrem =  -182.98276609537194
lowest likelihood = -13106.05660288389
ratio
12927.

-181.17153342213453

In [39]:
# init posterior avg at the jmax iteration
lafmn = np.zeros(4)
l2fmn = np.zeros(4)
lahfmn = 0

Hfmnfin = 0

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lafmn += walkers_fmn[i] * np.exp(llfmn[i])
    l2fmn += (walkers_fmn[i]**2) * np.exp(llfmn[i])
    lahfmn += llfmn[i]*np.exp(llfmn[i])
    
lafmn *= np.exp(logwt)
l2fmn *= np.exp(logwt)
lahfmn *= np.exp(logwt)

#final posterior calculation
postfinfmn = lpostavgfmn+lafmn
post2mfinfmn = post2mavgfmn+l2fmn

# dividing the posterior by the evidence
normpostfinfmn = postfinfmn/np.exp(LogfinalZfmn)
normpost2mfinfmn = post2mfinfmn/np.exp(LogfinalZfmn)

# information final calculation
Hfmnfin = prevHfmn + lahfmn
Hfmnfin = Hfmnfin/np.exp(LogfinalZfmn) - LogfinalZfmn
logZerrfmn = np.sqrt(Hfmnfin/npoints)

# variance and eror of posterior parameters
varfmn = normpost2mfinfmn - normpostfinfmn**2
stddevfmn = np.sqrt(varfmn)

# printing information
print("Posterior theta = "+ str(normpostfinfmn)+" Error = "+str(stddevfmn))
print("ln(Evidence) = "+str(LogfinalZfmn)+" +/- "+str(logZerrfmn))

Posterior theta = [ 0.23566931  0.90377094  0.         -0.03475965] Error = [0.05132585 0.09669251 0.         0.0214461 ]
ln(Evidence) = -181.17153342213453 +/- 0.13434442074831837


In [40]:
# seeing the difference of the Evidence
print("log(Zpc) - log(Zfc) = "+ str(LogfinalZpc - LogfinalZfc )+" +/- " +str(logZerrfc + logZerrpc))

log(Zpc) - log(Zfc) = 4.6764347596645734 +/- 0.22968256599224837


In [41]:
# Model probability of the models considered
mpc = np.exp(LogfinalZpc)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))
mfc = np.exp(LogfinalZfc)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))
mpmn = np.exp(LogfinalZpmn)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))
mfmn = np.exp(LogfinalZfmn)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc)+np.exp(LogfinalZfmn)+np.exp(LogfinalZpmn))

print("Model prob for Pull clean = " + str(mpc)[:5])
print("Model prob for free clean = " + str(mfc)[:5])
print("Model prob for Pull mn = " + str(mpmn)[:5])
print("Model prob for free mn = " + str(mfmn)[:5])

Model prob for Pull clean = 0.902
Model prob for free clean = 0.008
Model prob for Pull mn = 0.081
Model prob for free mn = 0.007


In [42]:
# ratio of evidences considered
np.exp(LogfinalZpc)/(np.exp(LogfinalZfc))

107.38653047785313

# Send text of data to a .txt file.

In [73]:
otpt = []
header = "========================================================================================================== \n"
header +="                                    RESULTS FROM NESTED                                            \n"
header +="========================================================================================================== \n"
otpt.append(header)
lin1 = ','.join(str(i) for i in p_data)
lin1 = "The raw position data =  ["+lin1+"]  \n\n\n"
otpt.append(lin1)
linearr = [0,0,0,0]
MODELS = ["(free, clean)","(pull, clean)","(free, mn)","(pull, mn)"]
MODELPROB = [mfc,mpc,mfmn,mpmn]
MODELZ = [LogfinalZfc,LogfinalZpc, LogfinalZfmn, LogfinalZpmn]
MODELZERR = [logZerrfc, logZerrpc, logZerrfmn, logZerrpmn]
MODELPARAM = [normpostfinfc, normpostfinpc, normpostfinfmn, normpostfinpmn]
MODELPARAMERR = [stddevfc, stddevpc, stddevfmn, stddevpmn]
MODELINFO = [Hfcfin, Hpcfin, Hfmnfin, Hpmnfin]
MODELMLE = [max(loglfc), max(loglpc), max(loglfmn), max(loglpmn)]
for i in range(0,4):
    linearr[i] = f"Model %d  %s  with Model Prob = %.3f \n"%(i+1,MODELS[i],MODELPROB[i])
    linearr[i] += f"ln Evidence = %.3f  +/- %.5f \n"%(MODELZ[i], MODELZERR[i])
    linearr[i] += f"Posterior Paramters \n"
    linearr[i] += f" 1. Diffusion Coefficient D_0 = %.3f  +/-  %.5f \n"%(MODELPARAM[i][0],MODELPARAMERR[i][0])
    linearr[i] += f" 2. Exponent alpha            = %.3f  +/-  %.5f \n"%(MODELPARAM[i][1],MODELPARAMERR[i][1])
    linearr[i] += f" 3. Applied Force f           = %.3f  +/-  %.5f \n"%(MODELPARAM[i][2],MODELPARAMERR[i][2])
    linearr[i] += f" 4. Var of Measurement noise  = %.3f  +/-  %.5f \n"%(MODELPARAM[i][3],MODELPARAMERR[i][3])
    linearr[i] += f" Maximum Likelihood = %.3f \n"%(MODELMLE[i])
    linearr[i] += f" Information H = %.5f  \n\n"%(MODELINFO[i])
for i in range(0,4):
    otpt.append(linearr[i])
otpt
hp = f"Highest Model probability for the given data is for Model %s \n\n\n\n"%(MODELS[MODELZ.index(max(MODELZ))])
otpt.append(hp)

In [74]:
file = open("DataFromNS.txt", "w")
file.writelines(otpt)
file.close()

# irrelevant shit to test my moronic code

In [31]:
N=1000000
a = walkergen(1,theta_max_pmn,theta_min_pmn)[0]
lla = noisylikelihoodcalc(p_data, 1, *a)
a,lla

(array([54.82239909,  1.25804207,  0.67696522, 49.75126743]),
 -181140.7089370248)

In [49]:
a = walkergen(1,theta_max_pmn,theta_min_pmn)[0]
lla = noisylikelihoodcalc(p_data, 1, *a)
print("init_Walker = " + str(a))
print("init likelihood = " +str(lla))
t = MCMCwalker(p_data, 1, a, lla, N, 1)
step = 1
fp = t[1]
llf = noisylikelihoodcalc(p_data, 1, fp[0], fp[1], fp[2], fp[3])
# fp
# t[1]
llf,fp

init_Walker = [39.49752281 -1.84018763  0.5869377  35.31830072]
init likelihood = -416.8303434445235


(-276.00725922829736,
 array([ 0.02451343,  1.52059059, -0.08171865,  2.48382868]))

In [46]:
llt = noisylikelihoodcalc(p_data, 1, 0.2, 1,-0,0.5)
llt

-278.8998534070378